In [1]:
import pandas as pd
from datetime import timedelta
from shapely.geometry import Point, box
import geopandas as gpd

In [3]:
# 读取堤坝数据
df_dams = pd.read_csv('data/CA_df_dam.csv')
df_dams['date'] = pd.to_datetime(df_dams['date'])

# 定义时间窗口和空间窗口
time_window = timedelta(days=7)
space_buffer = 0.1

# 创建GeoDataFrame
geometry = [Point(xy) for xy in zip(df_dams['longitude'], df_dams['latitude'])]
gdf_dams = gpd.GeoDataFrame(df_dams, geometry=geometry)

In [5]:
# 合并堤坝数据
def merge_dam_data(gdf, time_window, space_buffer):
    merged_data = []
    
    while not gdf.empty:
        # 取第一个点作为参考点
        ref_point = gdf.iloc[0]
        ref_date = ref_point['date']
        
        # 创建时间和空间范围
        time_range = (gdf['date'] >= ref_date - time_window) & (gdf['date'] <= ref_date + time_window)
        space_range = gdf.geometry.apply(lambda x: x.within(ref_point.geometry.buffer(space_buffer)))
        
        # 获取在范围内的所有点
        in_range = gdf[time_range & space_range]
        
        # 创建合并后的空间范围和时间范围
        minx, miny, maxx, maxy = in_range.total_bounds
        merged_bbox = box(minx, miny, maxx, maxy)
        merged_start_date = in_range['date'].min()
        merged_end_date = in_range['date'].max()
        
        # 添加到合并后的数据
        merged_data.append({
            'bbox': merged_bbox,
            'start_date': merged_start_date,
            'end_date': merged_end_date
        })
        
        # 移除已经合并的点
        gdf = gdf.drop(in_range.index)
    
    return merged_data

In [7]:
merged_data = merge_dam_data(gdf_dams, time_window, space_buffer)

In [9]:
# 打印合并后的数据
for data in merged_data:
    print(data)

{'bbox': <POLYGON ((-118.136 35.933, -118.136 36.028, -118.206 36.028, -118.206 35.93...>, 'start_date': Timestamp('2020-08-22 00:00:00'), 'end_date': Timestamp('2020-08-22 00:00:00')}
{'bbox': <POLYGON ((-118.167 35.892, -118.167 35.927, -118.189 35.927, -118.189 35.89...>, 'start_date': Timestamp('2020-08-22 00:00:00'), 'end_date': Timestamp('2020-08-22 00:00:00')}
{'bbox': <POLYGON ((-118.136 35.933, -118.136 36.029, -118.202 36.029, -118.202 35.93...>, 'start_date': Timestamp('2016-07-15 00:00:00'), 'end_date': Timestamp('2016-07-15 00:00:00')}
{'bbox': <POLYGON ((-118.165 35.883, -118.165 35.932, -118.194 35.932, -118.194 35.88...>, 'start_date': Timestamp('2016-07-15 00:00:00'), 'end_date': Timestamp('2016-07-15 00:00:00')}
{'bbox': <POLYGON ((-118.132 35.933, -118.132 36.028, -118.197 36.028, -118.197 35.93...>, 'start_date': Timestamp('2013-04-20 00:00:00'), 'end_date': Timestamp('2013-04-20 00:00:00')}
{'bbox': <POLYGON ((-118.165 35.882, -118.165 35.927, -118.194 35.927, -118

In [11]:
def query_satellite_images(m2m, start_date, end_date, bbox):
    spatial_filter = {
        'filterType': 'mbr',
        'lowerLeft': {'longitude': bbox[0], 'latitude': bbox[1]},
        'upperRight': {'longitude': bbox[2], 'latitude': bbox[3]},
    }
    temporal_filter = {
        'startDate': start_date.strftime('%Y-%m-%d'),
        'endDate': end_date.strftime('%Y-%m-%d')
    }
    
    scenes = m2m.searchScenes(datasetName='landsat_etm_c2_l2', 
                              spatialFilter=spatial_filter, 
                              temporalFilter=temporal_filter,
                              maxCC=10, 
                              maxResults=100)
    return scenes

In [13]:
from M2M_package import api

# 执行查询和下载
username = 'zk2086'
password = 'Wohechunge2002@'
token = "Yfq@gleb2SsNf4VjuwFIh3@hmZV5DA_5UR1gWrx6Yl4hQa4p5YL7ds!fhkssxorJ"
m2m = api.M2M(username, password) # Instantiate the API class and it will prompt you to enter the credentials.


In [15]:

def query_and_download(merged_data, m2m):
    for data in merged_data:
        start_date = data['start_date']
        end_date = data['end_date']
        bbox = data['bbox'].bounds
        
        scenes = query_satellite_images(m2m, start_date, end_date, bbox)
        
        # 下载对应的场景
        if scenes:
            scene_ids = [scenes['results'][0]['entityId'] for scene in scenes]
            download_metadata = m2m.retrieveScenes(datasetName='landsat_etm_c2_l2', scenes=scenes)
            # 保存图像
            for scene in download_metadata:
                m2m.download(scene['entityId'])
        else:
            print("No scenes found for this query")

query_and_download(merged_data, m2m)

KeyboardInterrupt: 